# COGS 118A - Project Checkpoint

## Predicting Minimum Hamming Distance Between Word and Vocabulary List

# Names

- Peter Barnett
- William Lutz
- Ricardo Sedano

# Abstract 
Our goal is to predict the minimum number of character substitutions required to turn a given input string of alphabetical characters into an English dictionary word (into *any* English dictionary word). That is, to predict the hamming distance between a given input string and its nearest English word. This problem can be solved with brute force nearest-neighbor search, or with a BK-tree, but takes a considerable amount of time if one needs to make this prediction many times. So, the purpose of this project is to be able to generate approximations of this shortest distance more quickly. The words will come from the open source WordSet dictionary, the training inputs will be generated from random strings, and the training labels will be generated using the brute force search approach. The same process is used to create the test data. We will train our model on this data and its performance will be measured by computing the average error |predicted distance - true distance| across the test dataset.

# Background

Hamming distance is the measured difference between two strings of equal lengths. As a simple example, the two strings 'car' and 'cat' would have a Hamming distance of 1, meaning that they only differ by one character. This mathematical concept was first published in 1950 his paper 'Error detecting error correcting codes' <a name = "hamming"></a>[<sup>[1]</sup>](#hammingnote). </br> </br>

We see and acnowlegde how Hamming distance has multiple applications to solve difficult applications in science, for example, comparing genomic sequences <a name = "pinheiro"></a>[<sup>[2]</sup>](#pinheironote). Yet, while it has widely been used in DNA sequencing, error detection, and image processing, trying to find the minimum Hamming distance between an input and a large dataset is computationally expensive to calculate when iterating through many examples. 

For this project, our team aims to calculate the minimum Hamming distance from an input sting of text to dictionary entries of the same length. In this application, user creators can implement effecient algothitms for spell check, autocorrection<a name = "lalwari"></a>[<sup>[3]</sup>](#lalwarinote), text predition, and even word puzzle contruction. In the paper recently cited, the project team does not use Hamming distance for autocorrection. In a similar manner, they use the Levenshtein distance (a close relative to Hamming distance) in their project. This differs because as noted previously as well as in the recently sited paper, Hamming distance requires stings of the same length. That team concluded their algorithm is, in fact, suitable for autocorrection, specifically more so in web applications. Our project differs this by not honing on autocorrect, as well as not using the Levenshtein distance, but instead the orginal Hamming distance.

# Problem Statement

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).





# Data

UPDATED FROM PROPOSAL!

You should have obtained and cleaned (if necessary) data you will use for this project.

Please give the following infomration for each dataset you are using
- link/reference to obtain it
- description of the size of the dataset (# of variables, # of observations)
- what an observation consists of
- what some critical variables are, how they are represented
- any special handling, transformations, cleaning, etc you have done should be demonstrated here!


Our data for this is very simple. We source the English words from the open source WordSet dictionary, and we generate the training data ourselves by creating random strings with alphabet characters and using brute-force search to assign each string its corresponding minimum hamming distance label, relative to the WordSet dictionary. We may choose to generate our random strings according to a letter frequency distribution or a bigram or trigram, or otherwise by randomly mutating random dictionary word strings to produce realistic use-case examples.


# Proposed Solution

In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.


One solution to this problem is to use a multilayer perceptron with two hidden layers. The input layer consists of 27 x 10 = 270 nodes, representing the one-hot encoding of an alphabetical string with up to 10 letters (26 possible letters, plus 1 for the "empty letter" if the input word is less than 10 characters). The values in these nodes would be 1s if that letter is present in that position or 0s otherwise. The output layer would consist of one node, the value of which represents the predicted minimum hamming distance to the input word across all English dictionary words. The loss function would be the sum of squared distances between the predicted hamming distance and the actual hamming distance between the training set strings and their nearest English words. Since we can generate endless training data using the brute force search, we may not need to add a regularization term, but we may also add that. Then we use backpropogation to train the multilayer perception with that loss function, after splitting the training data up into batches for the loss function. The solution will be tested by measuring the average error rate |predicted distance - true distance| across the test set, and by measuring the average time the forward pass takes and comparing it to the average time a bk-tree takes to find the shortest distance with our dataset.

# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

In order to evaluate the accuracy of our model we will split our dataset of randomly generated strings into a test and training dataset. For the test dataset, we will use tradition methods to calculate the Hamming distance for each string in the dataset. Our metric will be calculating the average error between the predicted Hamming distance and the actual value. Our model will be considered accurate if it is both able to identify the correct Hamming distance more than 75% of the time and it is computationally more efficient than traditional Hamming calculations on the same dataset.



# Preliminary results

We've tried a few different configurations with the multilayer perceptron model -- varying the learning rate, the number of hidden layers, and the number of hidden nodes per layer.
We have not formalized this yet -- we will create tables and graphs showing the performance with these varied hyper-parameters soon.
For best average loss on the test set (MSE loss) we've seen a wide range of configurations achieve around 0.85, which roughly corresponds to an average hamming distance prediction
   error of 0.92 across our dataset. This is nowhere near an optimal solution, but it does illustrate that an MLP can learn at least some of the structure of the problem in order
   to predict the hamming distance between an input word and the vocabulary set. MLP might not be the right tool to get an accurate prediction. We'll have to do a more thorough and
   systematic hyper-parameter search with suitable error metric evaluations to tell.
   
We will soon interpret the floating-point hamming distance prediction of this model after truncation and investigate the performance of this categorical classifier using error metrics.
We haven't yet performed model selection. And we haven't yet tried using other supervised learning models to solve this problem.

# Preliminary results -- Code

In the below code, we had already used the hamming_distance, build_bk_tree, and min_hamming_distance functions to create the training and test data (from the original dictionary of English words), which is in the file output.csv

In [ ]:
import pybktree, string, random
from bktree import Tree
import torch
import torch.nn as nn
import torch.optim as optim
import csv
from sklearn.model_selection import train_test_split

WORD_LENGTH = 6

word_list = []
with open('frequency-alpha-alldicts.txt') as f:
    next(f)  # skip the first line (header)
    for line in f:
        word = line.split()[1]  # get the second column, which is the word
        if len(word) == WORD_LENGTH:
            word_list.append(word.lower())

def hamming_distance(s1, s2):
    return sum(ch1 != ch2 for ch1, ch2 in zip(s1, s2))

def build_bk_tree(word_list):
    tree = pybktree.BKTree(hamming_distance, word_list)
    return tree

def min_hamming_distance(input_word, tree):
    results = tree.find(input_word, 1)
    if results:
        return results[0][0]
    results = tree.find(input_word, 2)
    if results:
        return results[0][0]
    results = tree.find(input_word, 3)
    if results:
        return results[0][0]
    return float('inf')

word_tree = build_bk_tree(word_list)

# Used to build dataset of near-words (some of which will be actual words)
def mutate_word(word):
    mutation_count = random.randint(0, 3)
    mutated_indices = random.sample(range(len(word)), mutation_count)
    mutated_chars = [random.choice(string.ascii_lowercase) for _ in range(mutation_count)]
    mutated_word = list(word)
    for index, char in zip(mutated_indices, mutated_chars):
        mutated_word[index] = char
    return ''.join(mutated_word)

# X -- the input data
dataset_strings = []

# Y -- the labels
dataset_dists = []

with open('output.csv', mode='r') as f:
    reader = csv.reader(f)
    for row in reader:
        dataset_strings.append(row[0])
        dataset_dists.append(int(row[1]))

data = list(zip(dataset_strings,dataset_dists))

# Split data into train and test datasets
train_data, test_data = train_test_split(data, test_size=0.2)

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        #self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(-1, 6*26) # Flatten the input tensor to (batch_size, 6*26)
        x = torch.relu(self.fc1(x))
        #x = torch.relu(self.fc2(x)) # Add an additional hidden layer
        x = self.fc3(x)
        return x

# Define hyperparameters
#INPUT_SIZE = 26 # Each character is one-hot encoded as a vector of length 26 (lowercase letters)
HIDDEN_SIZE = 1000
OUTPUT_SIZE = 1 # Predicting a scalar output (minimum hamming distance)
LEARNING_RATE = 0.00001
NUM_EPOCHS = 200

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

# Prepare the data
def one_hot_encode(s):
    x = torch.zeros(len(s), 26)
    for i, c in enumerate(s):
        x[i][ord(c) - 97] = 1
    return x

train_data = [(one_hot_encode(s), d) for s, d in train_data] # One-hot encode input strings
train_loader = torch.utils.data.DataLoader(train_data, batch_size=2048, shuffle=True)

test_data = [(one_hot_encode(s), d) for s, d in test_data] # One-hot encode input strings
test_loader = torch.utils.data.DataLoader(test_data, batch_size=2048, shuffle=False)

# Initialize the model and optimizer
model = MLP(26*6, HIDDEN_SIZE, OUTPUT_SIZE).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Define the loss function (mean squared error)
criterion = nn.MSELoss()

# Train the model
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch %d: loss=%.4f' % (epoch+1, running_loss/len(train_loader)))

# Test the model
total_loss = 0.0
with torch.no_grad():
   for inputs, labels in test_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs.float())
      loss = criterion(outputs, labels.float())
      total_loss += loss.item()

avg_loss = total_loss / len(test_loader)
print('Average loss on test set:', avg_loss)

# Ethics & Privacy

There are little to no obvious ethics & privacy concerns that arise from our project. The process highlighted in the project computes a numerical value (hamming distance), which is not involving other data, especially excluding personal data. Seeing as this process takes a string and compares to an established word, we could see a potential algorithm that creates text dialogue from these words. This connects to artificial intellegence; AI may be able to generate converstations with users online, which could involve ethics issues in spontaneous text production or privacy issues if AI creates upon platforms asking for personal information.

# Team Expectations 

Each project team member has agreed to and is expected to: 
* Attend all-team meetings
* Remain attentive; communicate quickly and effectively
* Do work assigned to them
* Be respectful of each others' work
* Stay aware of deadlines and collaborate in favor of completing a comprehensive project

# Project Timeline Proposal

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 2/22  |  6 PM |  Brainstorm topics (all); exchange contact information | Project topic, Discuss ideal dataset(s) and ethics;Edit, finalize, and submit project proposal| 
| 3/6  |  7 PM |  Import & Wrangle Data | Discuss Wrangling and possible analytical approaches; Finalize wrangling/EDA; Begin programming for project | 
| 3/20  | 7 PM  | Continue programming for project | Discuss/edit project code; Draft results/conclusion/discussion   |
| 3/22  | Before 11:59 PM  | Discuss/edit full project; Complete project | Turn in Final Project  |

# Footnotes
<a name="hammingnote"></a>1.[^](#hamming): Hamming, R. W. (1950) Error detecting and error correcting codes. *Bell Systems Technical Journal, 29(2)*. https://ieeexplore.ieee.org/document/6772729<br> 
 

<a name="pinheironote"></a>2.[^](#pinheiro): Pinheiro, H.P.  Analysis of Variance for Hamming Distances Applied to Unbalanced Designs. (https://ime.unicamp.br/sites/default/files/pesquisa/relatorios/rp-2001-30.pdf).<br>

<a name="lalwaninote"></a>3.[^](#lalwani): Lalwani, M. (2014) Efficient Algorithm for Auto Correction Using n-gram Indexing *Nirma University, Ahmedabad, India* (https://www.researchgate.net/profile/Mahesh-Lalwani/publication/266886742_Efficient_Algorithm_for_Auto_Correction_Using_n-gram_Indexing/links/547ee25e0cf2d2200edeaf9f/Efficient-Algorithm-for-Auto-Correction-Using-n-gram-Indexing.pdf).<br>
